---
### Project : 하루시작 
### Description : 모든역의 승차인원과 하차인원을 알때 현재 시간대의 특정역에서의 평균 탑승객수를 계산하는 알고리즘 

    
### Author : Forrest Dpark
### Date : 2024.06.05 ~
### Detail : 
### Update: 
    *  2024.06.05 by pdg : 알고리즘 테스트
        - 기본 모듈 프로그램 만들기
    * 2024.06.12 by pdg : 알고리즘 처리 서버에 적용 
        - Import setting update 
        - 서버단에서 예측한 결과를 토대로 바로 탑승인원수를 예측할수있도록 함수 만들기 
        
---

In [3]:
#### Jupyter Basic Setting ####
import pandas as pd,numpy as np
import matplotlib.pyplot as plt,seaborn as sns  # 시각화
import warnings; warnings.filterwarnings('ignore')  # 경고 무시
import sys,os 
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
from Module.Functions import Service
# 기본 세팅
Service.plotSetting("seaborn-v0_8")


___## OS platform 한글 세팅완료 ## ___


> 간단한 알고리즘 문제  

아침 7시에서 8시 사이 쯤 에 도착한 B역 플랫폼에 다가오는 지하철..   
이 지하철은 ABC 단 세정거장만 운영하며 배차되는 지하철은 1호기 하나뿐이다.  
각 역에서 승차인원과 하차인원 을 안다면 현재 탑승 추정인원은 몇명인가?


In [51]:
import numpy as np
A_in = 10
A_out = 0
B_in = 12
B_out = 1
C_in = 0
C_out = A_in-A_out+B_in-B_out+C_in
inputPassenger = np.array([A_in,B_in,C_in])
outputPassenger = np.array([A_out,B_out,C_out])
A_present = A_in -A_out
B_present = A_present + B_in -B_out
C_present = B_present +C_in- C_out
stations = ['상일동', '고덕','명일','굽은다리','길동','강동']
def presentPassengerCalc(inputPassenger, outputPassenger):
    # 3개의 역에 1개의 지하철만 배차 된다고 가정  
    print("present passagenr number Printing ")
    print("역별 승차인원 : ",*inputPassenger)
    print("역별 하차인원 : ",*outputPassenger)
    diff_arr = inputPassenger - outputPassenger
    print("변동인원 :",*diff_arr)
    A_present = diff_arr[0]
    B_present = A_present + diff_arr[1]
    C_present = B_present +diff_arr[2]
    print("현재 추정 탑승인원 :",A_present,B_present,C_present)
    
# presentPassengerCalc(inputPassenger,outputPassenger)
def stationPrediction_one(stations):
    import pandas as pd
     # 3개의 역이아니라 n 개의 역의 승하차 정보를 알고있을때 선택한 역의 현재 추정 탑승인원 추정
    # n = len(stations) 
    # print(np.arange(1,n))
    input_pasasengers_rand = []
    output_pasasengers_rand = []
    presentPassenger= np.zeros(shape=(len(stations)), dtype=int)
    for i,station in enumerate(stations):
            input_pasasengers_rand.append(np.random.randint(1,100) if i !=len(stations)-1 else 0)
            output_pasasengers_rand.append(np.random.randint(1,100) if i !=0 else 0)
            if i >0:
                
                presentPassenger[i] = presentPassenger[i-1] +input_pasasengers_rand[i]-output_pasasengers_rand[i]
            else:
                presentPassenger[i] = input_pasasengers_rand[i]
            # print(station, f"역 => 승차: {input_pasasengers_rand[i]} ,하차 :{output_pasasengers_rand[i]}")
            # print('현재탑승인원 : ',presentPassenger)
    #역별 변동인원
    diff_arr = np.asarray(input_pasasengers_rand) - np.asarray(output_pasasengers_rand)

    print(diff_arr)
    result = pd.DataFrame(
        {
            '역명': stations,
            '승차인원': input_pasasengers_rand,
            '하차인원': output_pasasengers_rand,
            '변동인원': diff_arr,
            '탑승자수': presentPassenger
        }
    )
    return result
stationPrediction_one(stations)
    


상일동 역 => 승차: 28 ,하차 :0
현재탑승인원 :  [28  0  0  0  0  0]
고덕 역 => 승차: 91 ,하차 :68
현재탑승인원 :  [28 51  0  0  0  0]
명일 역 => 승차: 16 ,하차 :36
현재탑승인원 :  [28 51 31  0  0  0]
굽은다리 역 => 승차: 32 ,하차 :60
현재탑승인원 :  [28 51 31  3  0  0]
길동 역 => 승차: 88 ,하차 :11
현재탑승인원 :  [28 51 31  3 80  0]
강동 역 => 승차: 0 ,하차 :4
현재탑승인원 :  [28 51 31  3 80 76]
[ 28  23 -20 -28  77  -4]


역명  승차인원  하차인원  변동인원  탑승자수
0   상일동    28     0    28    28
1    고덕    91    68    23    51
2    명일    16    36   -20    31
3  굽은다리    32    60   -28     3
4    길동    88    11    77    80
5    강동     0     4    -4    76

In [68]:
## n개의 지하철이 배차되었을때 평균 탑승객수 계산 
def currentPassengerCalc(stations,pass_in,pass_out,dispached_subway_number):
    import pandas as pd
    # 승하차 정보 없을때 랜덤 승하차 인원 데이터 생성 
    if pass_in ==[] and pass_out ==[]:
        pass_in = np.zeros(shape=(len(stations)), dtype=int)
        pass_out = np.zeros(shape=(len(stations)), dtype=int)
        presentPassenger= np.zeros(shape=(len(stations)), dtype=int)
        for i,station in enumerate(stations):
                pass_in[i]= np.random.randint(1,100) if i !=len(stations)-1 else 0
            
                if i >0:
                    pass_out[i]= np.random.randint(1,presentPassenger[i-1])
                    presentPassenger[i] = presentPassenger[i-1] +pass_in[i]-pass_out[i]
                else:
                    presentPassenger[i] = pass_in[i]
                # print(station, f"역 => 승차: {input_pasasengers_rand[i]} ,하차 :{output_pasasengers_rand[i]}")
                # print('현재탑승인원 : ',presentPassenger)
    #역별 변동인원
    
    diff_arr = np.asarray(pass_in) - np.asarray(pass_out)

    print(f"{dispached_subway_number}개 지하철이 배차되었을때 ")
    result = pd.DataFrame(
        {
            '역명': stations,
            '승차인원': pass_in,
            '하차인원': pass_out,
            '변동인원': diff_arr,
            '탑승자수': presentPassenger,
            '배차당탑승자수': presentPassenger/dispached_subway_number,
            '량당빈좌석수' :42 -(presentPassenger/dispached_subway_number) #42 개 6*7 노약자 제외 
        }
    )
    return result
currentPassengerCalc(stations,[],[],5)
    


5개 지하철이 배차되었을때 


역명  승차인원  하차인원  변동인원  탑승자수  배차당탑승자수  량당빈좌석수
0   상일동    25     0    25    25      5.0    37.0
1    고덕    63    22    41    66     13.2    28.8
2    명일    19    57   -38    28      5.6    36.4
3  굽은다리    82     6    76   104     20.8    21.2
4    길동    28    46   -18    86     17.2    24.8
5    강동     0    27   -27    59     11.8    30.2